# Distance between CoCs
A factory can be placed in a CoC and serves two functions:

1. Provide shelter to people experiencing homelessness (PEH) in that CoC.
2. Serve as a source for other temporary housing units (THUs) that can be deployed to other CoCs.

To determine the optimal deployment of factories, we require the distance between each pair of CoCs; from this distance we can calculate a transportation cost that will be used in the optimization framework to strategically deploy THUs and factories. In this work, we consider two ways to calculate the distance between CoCs:

1. Haversine distance
2. Road network distance

## Haversine distance
The hversine distance determines the orthodromic distance between two points on a sphere. Given the latitude $\phi$ and longitude $\lambda$ of two points, we can calculate the distance according to:
$$
d = 2r \sin^{-1}\left( \sqrt \frac{1 - \cos(\Delta \phi) + \cos(\phi_1) \cos(\phi_2)(1-\cos(\Delta \lambda))}{2} \right)
$$
where $r$ is the radius of the Earth (approximately 6,370 km).

In [3]:
import math
# Earth radius R in miles
R = 3959
def calc_haversine_dist(lat1, long1, lat2, long2):
    delta_latitude = lat2 - lat1
    delta_longitude = long2 - long1
    return 2*R*math.asin(math.sqrt((1 - math.cos(delta_latitude) + math.cos(lat1)*math.cos(lat2)*(1 - math.cos(delta_longitude)))/2))


In [ ]:
import geopandas as gpd

gdf = gpd.read_file('AK_500.shp')
gdf.info()
gdf.head()

# Calculate centroids
centroids = gdf.geometry.centroid

# Extract x (longitude) and y (latitude)
gdf['centroid_lon'] = centroids.x
gdf['centroid_lat'] = centroids.y

# Preview the result
print(gdf[['centroid_lon', 'centroid_lat']].head())
